# Parte 1: Análisis Exploratorio y Preprocesamiento

**Taller 3 - Clasificación de Imágenes Médicas**

---

## Objetivos
1. Cargar y visualizar ejemplos de ambas clases (NORMAL vs PNEUMONIA)
2. Analizar distribución de clases y tamaño de imágenes
3. Implementar pipeline de preprocesamiento:
   - Normalización de tamaño
   - Ecualización de histograma (CLAHE)
   - Segmentación de región de interés (opcional)


## 1. Configuración e Imports


In [ ]:
# Imports estándar
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

# Agregar src al path
sys.path.insert(0, os.path.abspath('..'))

# Imports de nuestros módulos
from src.data_loader import load_image_paths, split_by_set, get_class_distribution
from src.preprocessing import read_and_preprocess, get_image_sizes, apply_clahe, IMG_SIZE

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Imports completados correctamente ✓")


In [ ]:
# Configuración del dataset
DATA_DIR = "../data/chest_xray/chest_xray"  # Ajustar según ubicación del dataset

# Verificar que existe el directorio
if os.path.exists(DATA_DIR):
    print(f"✓ Dataset encontrado en: {DATA_DIR}")
else:
    print(f"✗ No se encontró el dataset en: {DATA_DIR}")
    print("  Por favor, descargue el dataset de Kaggle y ajuste la ruta.")


## 2. Carga de Datos

Cargamos las rutas de las imágenes y sus etiquetas usando la estructura del dataset:
```
chest_xray/
├── train/
│   ├── NORMAL/
│   └── PNEUMONIA/
├── val/
│   ├── NORMAL/
│   └── PNEUMONIA/
└── test/
    ├── NORMAL/
    └── PNEUMONIA/
```


In [ ]:
# Cargar todas las rutas y etiquetas
paths, labels = load_image_paths(DATA_DIR)

print(f"Total de imágenes cargadas: {len(paths)}")
print(f"\nEjemplo de rutas:")
for i in range(min(3, len(paths))):
    print(f"  {paths[i]}")
    print(f"    -> {labels[i]}")


In [ ]:
# Separar por conjuntos (train, val, test)
(paths_train, labels_train), (paths_val, labels_val), (paths_test, labels_test) = split_by_set(paths, labels)

print("Distribución por conjuntos:")
print(f"  Train: {len(paths_train)} imágenes")
print(f"  Val:   {len(paths_val)} imágenes")
print(f"  Test:  {len(paths_test)} imágenes")
print(f"  Total: {len(paths_train) + len(paths_val) + len(paths_test)} imágenes")


## 3. Análisis de Distribución de Clases

Es importante analizar el desbalance de clases, ya que esto afecta el entrenamiento de los modelos.


In [ ]:
# Crear DataFrame para análisis
df = pd.DataFrame({
    'path': paths,
    'split': [l[0] for l in labels],
    'class': [l[1] for l in labels]
})

print("Distribución general de clases:")
print(df['class'].value_counts())
print(f"\nRatio PNEUMONIA/NORMAL: {df['class'].value_counts()['PNEUMONIA'] / df['class'].value_counts()['NORMAL']:.2f}")


In [ ]:
# Distribución por split y clase
print("Distribución detallada por conjunto y clase:")
distribution = df.groupby(['split', 'class']).size().unstack(fill_value=0)
print(distribution)

# Calcular porcentajes
print("\nPorcentajes por conjunto:")
for split in ['train', 'val', 'test']:
    split_data = df[df['split'] == split]
    total = len(split_data)
    normal = len(split_data[split_data['class'] == 'NORMAL'])
    pneumonia = len(split_data[split_data['class'] == 'PNEUMONIA'])
    print(f"  {split}: NORMAL={normal/total*100:.1f}%, PNEUMONIA={pneumonia/total*100:.1f}%")


In [ ]:
# Visualización de la distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Distribución general
df['class'].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribución General de Clases', fontsize=14)
axes[0].set_xlabel('Clase')
axes[0].set_ylabel('Cantidad de imágenes')
axes[0].tick_params(axis='x', rotation=0)

# Añadir valores sobre las barras
for i, v in enumerate(df['class'].value_counts()):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

# Gráfico 2: Distribución por split
distribution.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'])
axes[1].set_title('Distribución por Conjunto', fontsize=14)
axes[1].set_xlabel('Conjunto')
axes[1].set_ylabel('Cantidad de imágenes')
axes[1].tick_params(axis='x', rotation=0)
axes[1].legend(title='Clase')

plt.tight_layout()
plt.savefig('../results/01_class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Observación: El dataset está desbalanceado con más casos de PNEUMONIA.")
print("   Esto debe considerarse en el entrenamiento de los modelos.")


## 4. Análisis de Tamaños de Imágenes

Analizamos la variabilidad en los tamaños originales de las imágenes para decidir el tamaño de redimensionamiento.


In [ ]:
# Obtener tamaños de una muestra de imágenes (para eficiencia)
sample_size = min(500, len(paths))
sample_indices = np.random.choice(len(paths), sample_size, replace=False)
sample_paths = [paths[i] for i in sample_indices]

print(f"Analizando tamaños de {sample_size} imágenes...")
sizes = get_image_sizes(sample_paths)

widths = [s[0] for s in sizes]
heights = [s[1] for s in sizes]

print(f"\nEstadísticas de tamaño:")
print(f"  Ancho  - Min: {min(widths)}, Max: {max(widths)}, Promedio: {np.mean(widths):.0f}, Mediana: {np.median(widths):.0f}")
print(f"  Alto   - Min: {min(heights)}, Max: {max(heights)}, Promedio: {np.mean(heights):.0f}, Mediana: {np.median(heights):.0f}")


In [ ]:
# Visualización de distribución de tamaños
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Histograma de anchos
axes[0].hist(widths, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(np.mean(widths), color='red', linestyle='--', label=f'Media: {np.mean(widths):.0f}')
axes[0].axvline(np.median(widths), color='orange', linestyle='--', label=f'Mediana: {np.median(widths):.0f}')
axes[0].set_title('Distribución de Anchos')
axes[0].set_xlabel('Ancho (píxeles)')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()

# Histograma de altos
axes[1].hist(heights, bins=30, color='coral', edgecolor='black', alpha=0.7)
axes[1].axvline(np.mean(heights), color='red', linestyle='--', label=f'Media: {np.mean(heights):.0f}')
axes[1].axvline(np.median(heights), color='orange', linestyle='--', label=f'Mediana: {np.median(heights):.0f}')
axes[1].set_title('Distribución de Altos')
axes[1].set_xlabel('Alto (píxeles)')
axes[1].set_ylabel('Frecuencia')
axes[1].legend()

# Scatter plot de dimensiones
axes[2].scatter(widths, heights, alpha=0.5, c='purple')
axes[2].set_title('Relación Ancho vs Alto')
axes[2].set_xlabel('Ancho (píxeles)')
axes[2].set_ylabel('Alto (píxeles)')
axes[2].axhline(256, color='red', linestyle='--', alpha=0.5)
axes[2].axvline(256, color='red', linestyle='--', alpha=0.5, label='Tamaño objetivo: 256x256')
axes[2].legend()

plt.tight_layout()
plt.savefig('../results/01_image_sizes.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💡 Decisión: Redimensionar todas las imágenes a {IMG_SIZE} para uniformidad.")


## 5. Visualización de Ejemplos por Clase

Visualizamos ejemplos representativos de ambas clases para entender las características visuales.


In [ ]:
# Seleccionar ejemplos de cada clase del conjunto de entrenamiento
normal_indices = [i for i, l in enumerate(labels_train) if l == 'NORMAL']
pneumonia_indices = [i for i, l in enumerate(labels_train) if l == 'PNEUMONIA']

n_examples = 4

fig, axes = plt.subplots(2, n_examples, figsize=(16, 8))

# Ejemplos NORMAL
sample_normal = np.random.choice(normal_indices, n_examples, replace=False)
for i, idx in enumerate(sample_normal):
    img = cv2.imread(paths_train[idx], cv2.IMREAD_GRAYSCALE)
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'NORMAL\n{img.shape[1]}x{img.shape[0]}')
    axes[0, i].axis('off')

# Ejemplos PNEUMONIA
sample_pneumonia = np.random.choice(pneumonia_indices, n_examples, replace=False)
for i, idx in enumerate(sample_pneumonia):
    img = cv2.imread(paths_train[idx], cv2.IMREAD_GRAYSCALE)
    axes[1, i].imshow(img, cmap='gray')
    axes[1, i].set_title(f'PNEUMONIA\n{img.shape[1]}x{img.shape[0]}')
    axes[1, i].axis('off')

plt.suptitle('Ejemplos de Rayos X de Tórax - Imágenes Originales', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/01_examples_original.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Observaciones:")
print("   - Las imágenes tienen diferentes tamaños y contrastes")
print("   - Los casos de neumonía muestran opacidades en los pulmones")
print("   - Es necesario normalizar el contraste para mejor extracción de características")


## 6. Pipeline de Preprocesamiento

Implementamos un pipeline de preprocesamiento que incluye:
1. **Redimensionamiento**: A tamaño uniforme (256x256)
2. **CLAHE** (Contrast Limited Adaptive Histogram Equalization): Mejora el contraste local
3. **Normalización**: Escalar valores al rango [0, 1]

### ¿Por qué CLAHE?
- Las radiografías tienen bajo contraste natural
- CLAHE mejora el contraste de forma adaptativa sin sobre-amplificar el ruido
- Es especialmente útil para imágenes médicas


In [ ]:
# Demostración del pipeline de preprocesamiento
sample_path = paths_train[normal_indices[0]]

# Cargar imagen original
img_original = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)
print(f"Imagen original: {img_original.shape}")

# Aplicar preprocesamiento completo
img_preprocessed = read_and_preprocess(sample_path, img_size=IMG_SIZE, apply_clahe=True)
print(f"Imagen preprocesada: {img_preprocessed.shape}")
print(f"Rango de valores: [{img_preprocessed.min():.3f}, {img_preprocessed.max():.3f}]")


In [ ]:
# Comparación visual: Original vs Preprocesada
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Seleccionar 4 imágenes aleatorias
sample_indices = np.random.choice(len(paths_train), 4, replace=False)

for i, idx in enumerate(sample_indices):
    # Original
    img_orig = cv2.imread(paths_train[idx], cv2.IMREAD_GRAYSCALE)
    axes[0, i].imshow(img_orig, cmap='gray')
    axes[0, i].set_title(f'Original\n{labels_train[idx]}')
    axes[0, i].axis('off')
    
    # Preprocesada
    img_prep = read_and_preprocess(paths_train[idx], apply_clahe=True)
    axes[1, i].imshow(img_prep, cmap='gray')
    axes[1, i].set_title(f'Preprocesada (CLAHE)\n{IMG_SIZE}')
    axes[1, i].axis('off')

plt.suptitle('Comparación: Imágenes Originales vs Preprocesadas', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/01_preprocessing_comparison.png', dpi=150, bbox_inches='tight')
plt.show()


### 6.1 Análisis del efecto de CLAHE en los histogramas


In [ ]:
# Comparación de histogramas antes y después de CLAHE
sample_path = paths_train[np.random.choice(len(paths_train))]

# Cargar y procesar
img_original = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)
img_resized = cv2.resize(img_original, IMG_SIZE)
img_clahe = read_and_preprocess(sample_path, apply_clahe=True)
img_clahe_uint8 = (img_clahe * 255).astype(np.uint8)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Fila 1: Imágenes
axes[0, 0].imshow(img_original, cmap='gray')
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_resized, cmap='gray')
axes[0, 1].set_title(f'Redimensionada ({IMG_SIZE})')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_clahe, cmap='gray')
axes[0, 2].set_title('Con CLAHE')
axes[0, 2].axis('off')

# Fila 2: Histogramas
axes[1, 0].hist(img_original.flatten(), bins=256, color='steelblue', alpha=0.7)
axes[1, 0].set_title('Histograma Original')
axes[1, 0].set_xlabel('Intensidad')
axes[1, 0].set_ylabel('Frecuencia')

axes[1, 1].hist(img_resized.flatten(), bins=256, color='coral', alpha=0.7)
axes[1, 1].set_title('Histograma Redimensionada')
axes[1, 1].set_xlabel('Intensidad')
axes[1, 1].set_ylabel('Frecuencia')

axes[1, 2].hist(img_clahe_uint8.flatten(), bins=256, color='green', alpha=0.7)
axes[1, 2].set_title('Histograma con CLAHE')
axes[1, 2].set_xlabel('Intensidad')
axes[1, 2].set_ylabel('Frecuencia')

plt.suptitle('Efecto del Preprocesamiento en la Distribución de Intensidades', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/01_histogram_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Observación: CLAHE distribuye mejor los niveles de gris,")
print("   mejorando el contraste y haciendo más visibles los detalles anatómicos.")


### 6.2 Exploración de parámetros de CLAHE


In [ ]:
# Explorar diferentes parámetros de CLAHE
sample_path = paths_train[np.random.choice(len(paths_train))]
img = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, IMG_SIZE)

# Diferentes configuraciones de CLAHE
clip_limits = [1.0, 2.0, 4.0]
tile_sizes = [(4, 4), (8, 8), (16, 16)]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i, clip in enumerate(clip_limits):
    for j, tile in enumerate(tile_sizes):
        clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=tile)
        img_clahe = clahe.apply(img)
        
        axes[i, j].imshow(img_clahe, cmap='gray')
        axes[i, j].set_title(f'clipLimit={clip}, tile={tile}')
        axes[i, j].axis('off')

plt.suptitle('Exploración de Parámetros CLAHE', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/01_clahe_parameters.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Parámetros seleccionados: clipLimit=2.0, tileGridSize=(8,8)")
print("   - Balance entre mejora de contraste y preservación de detalles")
print("   - Evita artefactos de sobre-amplificación")


## 7. Segmentación de Región de Interés (Opcional)

Exploramos la segmentación usando umbralización de Otsu para aislar la región pulmonar.


In [ ]:
from src.descriptors.shape import segmentar

# Demostración de segmentación
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

sample_indices = np.random.choice(len(paths_train), 4, replace=False)

for i, idx in enumerate(sample_indices):
    img = read_and_preprocess(paths_train[idx], apply_clahe=True)
    mask = segmentar(img)
    
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'Imagen Preprocesada\n{labels_train[idx]}')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(mask, cmap='gray')
    axes[1, i].set_title('Máscara (Otsu)')
    axes[1, i].axis('off')

plt.suptitle('Segmentación con Umbralización de Otsu', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/01_segmentation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Nota: La segmentación con Otsu es útil para:")
print("   - Calcular descriptores de contorno")
print("   - Descriptores de Fourier del borde")
print("   - Métricas geométricas (área, perímetro, etc.)")


## 8. Resumen del Pipeline de Preprocesamiento

### Decisiones tomadas:

| Parámetro | Valor | Justificación |
|-----------|-------|---------------|
| Tamaño de imagen | 256×256 | Balance entre detalle y eficiencia computacional |
| CLAHE clipLimit | 2.0 | Mejora contraste sin artefactos |
| CLAHE tileGridSize | (8, 8) | Adaptación local adecuada |
| Normalización | [0, 1] | Estándar para procesamiento |


In [ ]:
# Resumen del pipeline
print("="*60)
print("PIPELINE DE PREPROCESAMIENTO")
print("="*60)
print("""
1. CARGA DE IMAGEN
   └── cv2.imread(path, IMREAD_GRAYSCALE)
   
2. REDIMENSIONAMIENTO
   └── cv2.resize(img, (256, 256), interpolation=INTER_AREA)
   
3. MEJORA DE CONTRASTE (CLAHE)
   └── cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
   
4. NORMALIZACIÓN
   └── img.astype(float32) / 255.0  →  Rango [0, 1]
   
5. (OPCIONAL) SEGMENTACIÓN
   └── cv2.threshold(img, THRESH_BINARY + THRESH_OTSU)
""")
print("="*60)


## 9. Guardado de Resultados


In [ ]:
# Crear directorio de resultados si no existe
os.makedirs('../results', exist_ok=True)

# Guardar estadísticas del dataset
stats = {
    'total_images': len(paths),
    'train_images': len(paths_train),
    'val_images': len(paths_val),
    'test_images': len(paths_test),
    'class_distribution': df['class'].value_counts().to_dict(),
    'preprocessing': {
        'img_size': list(IMG_SIZE),
        'clahe_clip_limit': 2.0,
        'clahe_tile_size': [8, 8]
    }
}

import json
with open('../results/01_dataset_stats.json', 'w') as f:
    json.dump(stats, f, indent=2)

print("✓ Resultados guardados en ../results/")
print("  - 01_class_distribution.png")
print("  - 01_image_sizes.png")
print("  - 01_examples_original.png")
print("  - 01_preprocessing_comparison.png")
print("  - 01_histogram_comparison.png")
print("  - 01_clahe_parameters.png")
print("  - 01_segmentation.png")
print("  - 01_dataset_stats.json")


## 10. Conclusiones

### Hallazgos principales:

1. **Desbalance de clases**: El dataset tiene significativamente más casos de PNEUMONIA que NORMAL. Esto debe considerarse durante el entrenamiento (pesos de clase, oversampling, etc.).

2. **Variabilidad de tamaños**: Las imágenes originales tienen tamaños muy diversos. La estandarización a 256×256 es necesaria.

3. **Mejora de contraste**: CLAHE mejora significativamente la visibilidad de estructuras anatómicas en las radiografías.

4. **Pipeline robusto**: El preprocesamiento implementado prepara las imágenes de forma consistente para la extracción de características.

### Próximos pasos:
- Notebook 02: Extracción de descriptores clásicos (HOG, Hu, LBP, GLCM, Gabor, Fourier)
- Notebook 03: Clasificación y comparación de modelos
